In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import os
import glob
import pandas as pd

PATH = ""

In [2]:
PATH = ""
!ls {PATH}

fastai
labeled_text
models
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp
train
train.tsv
valid


In [3]:
df_train = pd.read_csv(f'{PATH}train.tsv', sep="\t")

In [4]:
df_test = pd.read_csv(f'{PATH}test.tsv',sep="\t")

In [5]:
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [131]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


# Language Model

In [6]:
#drop incomplete sentences in training set
def dropIncomplete (df, label="SentenceId", keepLocation=0):
    sentences = []
    for i in df[label].unique():  
        sentences.append(df[df[label] == i].reset_index(drop=True).loc[keepLocation])
    df_result = pd.DataFrame(sentences)
    return df_result

In [133]:
df_train = dropIncomplete(df_train)

In [134]:
df_test = dropIncomplete(df_test)

In [135]:
df_train = pd.DataFrame(df_train["Phrase"]).reset_index(drop=True)
df_test = pd.DataFrame(df_test["Phrase"]).reset_index(drop=True)

In [136]:
df_train.head()

,Phrase
0,A series of escapades demonstrating the adage ...
1,"This quiet , introspective and entertaining in..."
2,"Even fans of Ismail Merchant 's work , I suspe..."
3,A positively thrilling combination of ethnogra...
4,Aggressive self-glorification and a manipulati...


In [137]:
df_test.head()

,Phrase
0,An intermittently pleasing but mostly routine ...
1,Kidman is really the only thing that 's worth ...
2,Once you get into its rhythm ... the movie bec...
3,I kept wishing I was watching a documentary ab...
4,"Kinnear does n't aim for our sympathy , but ra..."


In [138]:
df_full = df_train.append(df_test).reset_index(drop=True)

In [139]:
df_full[df_full.shape[1]-10:]

,Phrase
11830,"At its worst , it implodes in a series of very..."
11831,Maybe I found the proceedings a little bit too...
11832,Moretti 's compelling anatomy of grief and the...
11833,Montias ... pumps a lot of energy into his nic...
11834,Not sweet enough to liven up its predictable s...
11835,"Nasty , ugly , pointless and depressing , even..."
11836,"With tightly organized efficiency , numerous f..."
11837,They should have called it Gutterball .
11838,"A long-winded , predictable scenario ."


In [140]:
df_full.shape

(11839, 1)

In [141]:
spacy_tok = spacy.load('en')

In [142]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [143]:
bs=64; bptt=70

In [144]:
df_train.shape, df_test.shape

((8529, 1), (3310, 1))

In [145]:
md = LanguageModelData.from_dataframes(path=PATH,field=TEXT,col="Phrase",train_df=df_train, val_df=df_test,
                                       test_df=df_test,bs=bs, bptt=bptt)

In [146]:
pickle.dump(TEXT, open(f'{PATH}TEXT.pkl','wb'))

In [147]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp/
train.tsv


In [148]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(42, 15364, 1, 193010)

In [149]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', '<', '>', 'eos', '.', 'the', ',', 'a', 'and', 'of', 'to']

In [150]:
TEXT.vocab.stoi['to']

11

In [151]:
em_sz = 200
nh = 500
nl = 3

In [152]:
opt_fn = partial(optim.Adam, betas = (0.7, 0.99))

In [153]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05, dropout=0.05,
                       wdrop=0.1, dropoute=0.02, dropouth=0.05)

In [154]:
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [155]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                            
    0      6.954403   6.299073  
    1      6.281553   5.876824                            
    2      5.920499   5.804518                            
    3      5.776692   5.743025                            
    4      5.606106   5.600349                            
    5      5.453188   5.556259                            
    6      5.349419   5.533326                            
    7      5.306063   5.470198                            
    8      5.203112   5.369964                            
    9      5.084493   5.375473                            
    10     4.960781   5.319035                            
    11     4.849896   5.346584                            
    12     4.762601   5.342232                            
    13     4.703944   5.341238                            
    14     4.658045   5.33422                             



[array([5.33422])]

In [156]:
learner.save_encoder('adam1_movieReview_enc')

In [157]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

epoch      trn_loss   val_loss                            
    0      4.678274   5.305937  
    1      4.665965   5.211305                            
    2      4.596527   5.224018                            
    3      4.497864   5.255579                            
    4      4.387083   5.296661                            
    5      4.299023   5.330986                            
    6      4.212508   5.342638                            
    7      4.142816   5.358443                            
    8      4.092514   5.374074                            
    9      4.052414   5.36959                             



[array([5.36959])]

In [158]:
learner.save_encoder('adam3_movieReview_10_enc')

In [159]:
learner.load_encoder('adam3_movieReview_10_enc')

# Test

In [160]:
#use our torchtext field to numericalize it so we can feed it to our language model.
m = learner.model
ss = """This is not something that I will have"""
s = [TEXT.preprocess(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

'this is not something that i will have'

In [161]:
#steps to test a language model
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [162]:
#Top 10 predictions
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

['been', 'a', 'seen', 'made', '.', 'enough', 'as', 'done', 'not', 'to']

In [163]:
#generate a bit more text
print(ss,"\n")
for i in range(20):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res, *_ =m(n[0].unsqueeze(0))
print("...")

This is not something that I will have 

been a good movie . < eos > the film is a lot of fun , but it 's not ...


# Sentiment

In [6]:
#Process training set data with sentiment label
df_trainWLabels = pd.read_csv("train.tsv", sep="\t")
#df_trainWLabels = dropIncomplete(df_trainWLabels)
df_trainWLabels.drop(["PhraseId", "SentenceId"], axis = 1,inplace=True)
df_trainWLabels.reset_index(inplace=True, drop=True)
df_trainWLabels.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,A series of escapades demonstrating the adage ...,2
2,A series,2
3,A,2
4,series,2


In [7]:
len(df_trainWLabels)

156060

In [8]:
def IntoLabeledFolders(df, folderPath, validRatio=0.3, contentIndex=0, labelIndex=1):
    count = 1
    L = len(df)
    cutPoint = int(round((1-validRatio)*L))

    for index, row in df.iterrows():
        phrase=row[contentIndex]
        label = row[labelIndex]
        if (count<cutPoint):
            filename = os.path.join(folderPath,"train", str(label), str(count))+".txt"
        else:
            filename = os.path.join(folderPath,"valid", str(label), str(count))+".txt"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename,"w") as f:
            f.write(phrase)
        count = count + 1

In [9]:
IntoLabeledFolders(df=df_trainWLabels, folderPath = PATH)

In [10]:
#Show labels ###Start here if already created labeled folders
labelList = !ls {PATH}train
labelList

['0', '1', '2', '3', '4']

In [11]:
#Count number of samples under each label
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}train/{i}
    labelCount.append(len(trn_files))
labelCount

[4886, 18496, 56471, 23074, 6314]

In [12]:
#Count number of samples under each label
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}valid/{i}
    labelCount.append(len(trn_files))
labelCount

[2186, 8777, 23111, 9853, 2892]

In [13]:
#Review sentences
label = 3
fileIndex = 40
FileList = !ls {PATH}train/{label}
sample = !cat {PATH}train/{label}/{FileList[fileIndex]}
sample

['every bit as fascinating']

In [47]:
class MultiLabeledMovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [32]:
MovieReviewLabel = data.Field(sequential=False)

In [33]:
TEXT = pickle.load(open(f"{PATH}TEXT.pkl","rb")) #open in read and bineary mode

In [34]:
PATH

''

In [35]:
splits = MultiLabeledMovieReview.splits(TEXT, MovieReviewLabel,PATH,train="train", test="valid")

In [36]:
t = splits[0].examples[5]

In [37]:
t.label, " ".join(t.text[:30])

('0',
 'a lame romantic comedy about an unsympathetic character and someone who would not likely be so stupid as to get')

In [38]:
bs = 64; bptt = 70

In [39]:
md2 = TextData.from_splits(PATH, splits, bs)

In [40]:
ls

fastai@
labeled_text/
models/
movie-review-sentiment-analysis-kernels-only.ipynb
movie-review-sentiment-analysis-kernels-only-transferred-model.ipynb
sampleSubmission.csv
test.tsv
TEXT_imdb.pkl
TEXT.pkl
tmp/
train/
train.tsv
valid/


In [41]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [42]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [43]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_movieReview_10_enc')

In [44]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [45]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                     
    0      1.091448   1.302075   0.440147  
    1      1.082849   1.288543   0.449714                     
    2      1.04546    1.247309   0.473057                     
    3      1.015726   1.23289    0.477534                      
    4      0.971378   1.233141   0.481612                      
    5      0.939413   1.209067   0.49132                       
    6      0.925436   1.206565   0.494397                      
    7      0.929338   1.193189   0.500282                      
    8      0.920551   1.198255   0.502415                      
    9      0.887644   1.189893   0.504714                      
    10     0.871363   1.185199   0.50661                       
    11     0.864495   1.182672   0.506517                      
    12     0.858642   1.189529   0.505425                      
    13     0.852789   1.178032   0.510983                      


[array([1.17803]), 0.5109833878707448]

# Positive vs Negative Sentiment

In [74]:
df_trainWLabels.shape

(8529, 2)

In [75]:
df_pvn = pd.DataFrame(df_trainWLabels[df_trainWLabels["Sentiment"]!=2])

In [76]:
df_pvn["Sentiment"] = df_pvn["Sentiment"].apply(lambda x: "pos" if x>2 else "neg")

In [77]:
df_pvn.shape

(6874, 2)

In [78]:
IntoLabeledFolders(df=df_pvn, folderPath = PATH)

In [79]:
#with the original labeled folders removed ##start here if existing folders
labelList = !ls {PATH}train
labelList

['neg', 'pos']

In [80]:
#Count number of samples under each label
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}train/{i}
    labelCount.append(len(trn_files))
labelCount

[2300, 2511]

In [81]:
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}valid/{i}
    labelCount.append(len(trn_files))
labelCount

[972, 1091]

In [82]:
#Review sentences
label = "pos"
fileIndex = 40
FileList = !ls {PATH}train/{label}
sample = !cat {PATH}train/{label}/{FileList[fileIndex]}
sample

["She 's all-powerful , a voice for a pop-cyber culture that feeds on her Bjorkness ."]

In [83]:
#Function to create torch text object
class PosNegMovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['pos','neg']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)
    
        

In [84]:
MovieReviewLabel = data.Field(sequential=False)

In [244]:
TEXT = pickle.load(open("TEXT.pkl","rb")) #open in read and bineary mode

In [245]:
splits = PosNegMovieReview.splits(TEXT, MovieReviewLabel, PATH,train="train", test="valid")

In [246]:
t = splits[0].examples[5]

In [247]:
t.label, " ".join(t.text[:30])

('pos',
 'yet this one makes up for in heart what it lacks in outright newness .')

In [248]:
bs = 64; bptt = 70

In [249]:
md2 = TextData.from_splits(PATH, splits, bs)

In [250]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [252]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_movieReview_10_enc')

In [253]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [254]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                  
    0      0.703548   0.677838   0.570314  
    1      0.666022   0.654203   0.61576                   
    2      0.655982   0.640636   0.629357                  
    3      0.634351   0.634049   0.637036                  
    4      0.633072   0.626915   0.630266                  
    5      0.614617   0.625476   0.63422                   
    6      0.610132   0.663844   0.596408                  
    7      0.586934   0.632855   0.633796                  
    8      0.601679   0.623809   0.63556                   
    9      0.577846   0.616723   0.6396                    
    10     0.580966   0.59687    0.664086                  
    11     0.559713   0.606161   0.659399                  
    12     0.560234   0.600168   0.662546                  
    13     0.537786   0.600744   0.6696                    



[array([0.60074]), 0.6695997044869831]

# Very positive vs very negative

In [15]:
df_trainWLabels.head()

,Phrase,Sentiment
0,A series of escapades demonstrating the adage ...,1
1,"This quiet , introspective and entertaining in...",4
2,"Even fans of Ismail Merchant 's work , I suspe...",1
3,A positively thrilling combination of ethnogra...,3
4,Aggressive self-glorification and a manipulati...,1


In [16]:
df_vneg = pd.DataFrame(df_trainWLabels[df_trainWLabels["Sentiment"]==0])

In [260]:
df_vneg.head()

,Phrase,Sentiment
31,"Hampered -- no , paralyzed -- by a self-indulg...",0
37,The movie 's progression into rambling incoher...,0
40,Tartakovsky 's team has some freakish powers o...,0
52,"Thanks largely to Williams , all the interesti...",0
53,"Comes across as a relic from a bygone era , an...",0


In [17]:
df_vpos = pd.DataFrame(df_trainWLabels[df_trainWLabels["Sentiment"]==4])
df_vpos.head()

,Phrase,Sentiment
1,"This quiet , introspective and entertaining in...",4
5,A comedy-drama of nearly epic proportions root...,4
12,The performances are an absolute joy .,4
20,"Best indie of the year , so far .",4
42,If a horror movie 's primary goal is to fright...,4


In [18]:
df_vpvn = df_vpos.append(df_vneg)

In [19]:
df_vpvn.shape

(2353, 2)

In [20]:
df_vpvn.head(10)

,Phrase,Sentiment
1,"This quiet , introspective and entertaining in...",4
5,A comedy-drama of nearly epic proportions root...,4
12,The performances are an absolute joy .,4
20,"Best indie of the year , so far .",4
42,If a horror movie 's primary goal is to fright...,4
48,Jones has tackled a meaty subject and drawn en...,4
49,Bloody Sunday has the grace to call for preven...,4
54,Perhaps it 's cliche to call the film ` refres...,4
55,Both lead performances are Oscar-size .,4
66,"A smart , provocative drama that does the near...",4


In [21]:
df_vpvn.tail(10)

,Phrase,Sentiment
8430,What starts off as a possible Argentine Americ...,0
8434,Staggeringly dreadful romance .,0
8445,This pathetic junk is barely an hour long .,0
8452,"At the one-hour mark , Herzog simply runs out ...",0
8468,A ragbag of cliches .,0
8480,Disgusting .,0
8485,Crossroads feels like a teenybopper Ed Wood fi...,0
8493,As exciting as all this exoticism might sound ...,0
8499,Every conceivable mistake a director could mak...,0
8523,The French director has turned out nearly 21\/...,0


In [22]:
df_vpvn = df_vpvn.sample(frac=1).reset_index(drop=True)
df_vpvn.head(10)

,Phrase,Sentiment
0,"Ramsay is clearly extraordinarily talented , a...",4
1,It zips along with B-movie verve while adding ...,4
2,"With Dirty Deeds , David Caesar has stepped in...",4
3,Certain to be distasteful to children and adul...,0
4,"The production design , score and choreography...",4
5,"Best indie of the year , so far .",4
6,"Sorority Boys , which is as bad at it is cruel...",0
7,"Good , solid storytelling .",4
8,It would take a complete moron to foul up a sc...,0
9,A grand fart coming from a director beginning ...,0


In [23]:
df_vpvn.shape

(2353, 2)

In [24]:
df_vpvn["Sentiment"] = df_vpvn["Sentiment"].apply(lambda x: "pos" if x>2 else "neg")

In [25]:
df_vpvn.head()

,Phrase,Sentiment
0,"Ramsay is clearly extraordinarily talented , a...",pos
1,It zips along with B-movie verve while adding ...,pos
2,"With Dirty Deeds , David Caesar has stepped in...",pos
3,Certain to be distasteful to children and adul...,neg
4,"The production design , score and choreography...",pos


In [26]:
IntoLabeledFolders(df=df_vpvn, folderPath = PATH)

In [27]:
labelList = !ls {PATH}train
labelList

['neg', 'pos']

In [28]:
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}train/{i}
    labelCount.append(len(trn_files))
labelCount

[756, 890]

In [29]:
labelCount = []
for i in labelList:
    trn_files = !ls {PATH}valid/{i}
    labelCount.append(len(trn_files))
labelCount

[316, 391]

In [30]:
MovieReviewLabel = data.Field(sequential=False)

In [57]:
TEXT = pickle.load(open("TEXT.pkl","rb")) #TEXT field for native NLP model

In [58]:
splits = MovieReview.splits(TEXT, MovieReviewLabel, PATH,train="train", test="valid")

In [59]:
bs = 64; bptt = 70

In [60]:
md2 = TextData.from_splits(PATH, splits, bs)

In [61]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [65]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_movieReview_10_enc') #native NLPmodel

In [66]:
m3.clip=25
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [67]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                  
    0      0.739345   0.691794   0.548284  
    1      0.695451   0.673288   0.589755                  
    2      0.682104   0.656902   0.598477                  
    3      0.662311   0.657581   0.592323                  
    4      0.657131   0.742148   0.548712                  
    5      0.641368   0.655737   0.61164                   
    6      0.624263   0.638172   0.631172                  
    7      0.604496   0.628725   0.635827                  
    8      0.609735   0.629723   0.625874                  
    9      0.590954   0.627067   0.620469                  
    10     0.592503   0.637078   0.626142                  
    11     0.571986   0.649532   0.632884                  
    12     0.563054   0.650176   0.636041                  
    13     0.545923   0.657449   0.631332                  


[array([0.65745]), 0.6313320508558457]

# Using a transfered model trained on IMDB data set

In [48]:
TEXT = pickle.load(open("TEXT_imdb.pkl","rb")) #TEXT field for IMDB NLP model
splits = MultiLabeledMovieReview.splits(TEXT, MovieReviewLabel, PATH,train="train", test="valid")

In [49]:
bs = 64; bptt = 70
md2 = TextData.from_splits(PATH, splits, bs)
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [50]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder('adam3_imdb_10_enc') #NLP model trained on IMDB dataset

In [51]:
m3.clip=25
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [52]:
# freeze/unfreeze and differential learning rate
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      1.139295   1.263814   0.432896  


epoch      trn_loss   val_loss   accuracy                     
    0      1.070088   1.167292   0.488185  


[array([1.16729]), 0.4881846352137183]

In [53]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                     
    0      1.052536   1.141739   0.509442  
    1      1.03386    1.147596   0.511788                      
    2      1.005644   1.210041   0.488842                      
    3      0.961546   1.168524   0.5204                        
    4      0.96673    1.154298   0.534447                      
    5      0.949491   1.172889   0.531779                      
    6      0.956332   1.20941    0.512285                      
    7      0.932446   1.18231    0.534402                      
    8      0.924049   1.2296     0.52465                       
    9      0.901105   1.169018   0.539088                      
    10     0.905614   1.207651   0.517092                      
    11     0.88481    1.211402   0.545079                      
    12     0.885735   1.226406   0.537763                      
    13     0.887049   1.188989   0.545634                      


[array([1.18899]), 0.5456343977969432]

In [54]:
#Not overfitting yet, continue training
m3.fit(lrs, 3, metrics=[accuracy], cycle_len=10, cycle_save_name='MovieReview')

epoch      trn_loss   val_loss   accuracy                      
    0      0.919168   1.194043   0.536609  
    1      0.892308   1.167078   0.558539                      
    2      0.886449   1.177978   0.542236                      
    3      0.889609   1.19618    0.541768                      
    4      0.852477   1.223079   0.535612                      
    5      0.862832   1.18232    0.541867                      
    6      0.84305    1.166159   0.552757                      
    7      0.832242   1.183578   0.551902                      
    8      0.816114   1.201127   0.555013                      
    9      0.833731   1.172674   0.55678                       
    10     0.854878   1.226053   0.541263                      
    11     0.864414   1.158709   0.535323                      
    12     0.832049   1.222211   0.538389                      
    13     0.820791   1.193392   0.545381                      
    14     0.813213   1.188233   0.530849                   

[array([1.18782]), 0.5568040695153538]